# Install dependencies 

In [1]:
!pip install -q transformers==4.3.2

!pip install -q torch

!pip install -q numpy
!pip install -q pandas

# Import libraries

In [2]:
import sys
import numpy as np
import pandas as pd
import os
os.chdir('..')
from GanBert import GanBert



# Set-up

Set gloabal variable for changing dataset.

Set data_name to the name of your dataset. This needs to correspond to a folder in /data/, which should be generated by the generate_data.ipynb notebook. num_classes manually needs to be set to the number of classes in your dataset.

- data_name:
    - "imdb"
    - "medical"


In [3]:
## gloabal variable for changing dataset.
## data_name possible values: "imdb", "medical"
data_name = "medical"

## Get data
- get the path for the labeled training data, 5, 10, 25, and 50 per label
- get the path for the unlabeled training data, 5, 10, 25, and 50 per label
- get the path for the test set

In [4]:
## method for convcerting the dataset to the right format for the ganbert model
def data_to_ganbert(data):    
    data = list(zip(data.iloc[:,0],data.iloc[:,1]))
    return data

In [5]:
## get the path for the labeled training data, 5, 10, 25, and 50 per label
data_path = 'data/'+data_name
files = os.listdir(data_path)
labeled_files = [data_path+"/"+file for file in files if "train_labeled" in file]

## get the path for the unlabeled training data, 5, 10, 25, and 50 per label
unlabeled = data_to_ganbert(pd.read_csv(f"data/{data_name}/train_unlabeled.csv")) 

## get the path for the test set
test = data_to_ganbert(pd.read_csv(f"data/{data_name}/test.csv"))

## Set hyper-parameters for Bert

For more details, see https://github.com/crux82/ganbert-pytorch/blob/main/GANBERT_pytorch.ipynb


In [6]:
# hyper parameters 
batch_size = 64
max_seq_length = 128
seed = 0
learning_rate = 5e-5
epochs=5

# create a data frame to store the results
results=pd.DataFrame(columns=["n_per_class", "accuracy"])

# Traning and evalueate the Bert classifier

In [7]:
## train and evaluate bert for each data set. 
for n_per_class in [5,10,25,50]:
    data_file = ""
    result = {"n_per_class":n_per_class}
    # create model
    ganbert = GanBert(batch_size=batch_size,max_seq_length= max_seq_length,epochs = epochs,
                      learning_rate_discriminator = learning_rate,learning_rate_generator = learning_rate,
                      print_each_n_step = 100,random_state = seed)
    ## find correct file
    for file in labeled_files:
        if f"data/{data_name}/train_labeled_{n_per_class}.csv" == file:
            data_file = file
            break
    print(data_file)
    labeled = data_to_ganbert(pd.read_csv(data_file))
    ## train and evaluate the model
    performance = ganbert.train(labeled,unlabeled, test)
    ## add to resutl data frame
    result["accuracy"] = performance
    results = results.append(result,ignore_index=True)

There are 1 GPU(s) available.
We will use the GPU: Tesla K80
data/medical/train_labeled_5.csv

======== Epoch 1 / 5 ========
Training...

  Average training loss generetor: 0.630
  Average training loss discriminator: 2.384
  Training epcoh took: 0:04:55

Running Test...
  Accuracy: 0.221
  Test Loss: 1.802
  Test took: 0:00:06

======== Epoch 2 / 5 ========
Training...

  Average training loss generetor: 0.742
  Average training loss discriminator: 1.299
  Training epcoh took: 0:04:55

Running Test...
  Accuracy: 0.254
  Test Loss: 2.736
  Test took: 0:00:06

======== Epoch 3 / 5 ========
Training...

  Average training loss generetor: 0.732
  Average training loss discriminator: 0.858
  Training epcoh took: 0:04:56

Running Test...
  Accuracy: 0.270
  Test Loss: 3.128
  Test took: 0:00:06

======== Epoch 4 / 5 ========
Training...

  Average training loss generetor: 0.716
  Average training loss discriminator: 0.755
  Training epcoh took: 0:04:56

Running Test...
  Accuracy: 0.266
  

In [8]:
## print the result
results

,n_per_class,accuracy
0,5.0,0.272321
1,10.0,0.459821
2,25.0,0.375000
3,50.0,0.549107


In [9]:
# write the result to file
if not os.path.exists('results'):
      os.mkdir('results')
result_path = f'results/{data_name}'
if not os.path.exists(result_path):
      os.mkdir(result_path)
results.to_csv(f"{result_path}/GanBert_results.csv", index=False)